In [2]:
# Cell 1
import requests
import pandas as pd
from datetime import date
from dateutil import parser, tz
from IPython.display import display

def fetch_json(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()


In [10]:
# Cell 2
today = date.today().isoformat()
sched = fetch_json(f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}")

if not sched.get("dates"):
    print(f"No games scheduled for {today}.")
else:
    for game in sched["dates"][0]["games"]:
        pk          = game["gamePk"]
        away        = game["teams"]["away"]["team"]["name"]
        away_score  = game["teams"]["away"]["score"]
        home        = game["teams"]["home"]["team"]["name"]
        home_score  = game["teams"]["home"]["score"]
        status      = game["status"]["detailedState"]
        print(f"{pk}: {away} {away_score} @ {home} {home_score} — {status}")


777230: Tampa Bay Rays 5 @ Minnesota Twins 1 — In Progress
777229: St. Louis Cardinals 3 @ Chicago Cubs 4 — In Progress
777231: Los Angeles Angels 2 @ Toronto Blue Jays 3 — In Progress
777228: Boston Red Sox 2 @ Washington Nationals 0 — In Progress
777233: Cincinnati Reds 0 @ Philadelphia Phillies 0 — In Progress
777234: Kansas City Royals 0 @ Arizona Diamondbacks 0 — In Progress
777237: Baltimore Orioles 0 @ Atlanta Braves 0 — In Progress
777241: New York Yankees 0 @ New York Mets 0 — In Progress
777238: Milwaukee Brewers 0 @ Miami Marlins 0 — In Progress
777240: Detroit Tigers 0 @ Cleveland Guardians 0 — Pre-Game
777221: Houston Astros 0 @ Los Angeles Dodgers 0 — Pre-Game


KeyError: 'score'

In [14]:
# Cell 3: enter a gamePk and display detailed stats with robust fallback

game_pk = int(input("Enter gamePk: "))

def show_linescore(df, label):
    print(f"{label} Linescore"); display(df)

def show_stats(rows, label):
    print(label); display(pd.DataFrame(rows))

# 1) Try live feed
resp = requests.get(f"https://statsapi.mlb.com/api/v1/game/{game_pk}/feed/live")
if resp.status_code == 200 and "liveData" in (payload := resp.json()):
    feed = payload["liveData"]
    
    # Live linescore
    ls = feed["linescore"]
    df_ls = (
        pd.DataFrame(ls["innings"])[["num","away","home"]]
          .rename(columns={"num":"Inning","away":"Away","home":"Home"})
    )
    df_ls["Away Total"] = ls["teams"]["away"]["runs"]
    df_ls["Home Total"] = ls["teams"]["home"]["runs"]
    show_linescore(df_ls, "Live")

    # Live batting
    bat_live = []
    for side in ("home","away"):
        team = feed["boxscore"]["teams"][side]
        for pid in team.get("batters",[]):
            p = feed["boxscore"]["players"].get(f"ID{pid}", {})
            stats = p.get("stats",{}).get("batting",{})
            bat_live.append({
                "Team": team["team"]["name"],
                "Player": p.get("person",{}).get("fullName",""),
                **{k:v for k,v in stats.items() if isinstance(v,(int,float,str))}
            })
    show_stats(bat_live, "Live Batting")

    # Live pitching
    pit_live = []
    for side in ("home","away"):
        team = feed["boxscore"]["teams"][side]
        for pid in team.get("pitchers",[]):
            p = feed["boxscore"]["players"].get(f"ID{pid}", {})
            stats = p.get("stats",{}).get("pitching",{})
            pit_live.append({
                "Team": team["team"]["name"],
                "Player": p.get("person",{}).get("fullName",""),
                **{k:v for k,v in stats.items() if isinstance(v,(int,float,str))}
            })
    show_stats(pit_live, "Live Pitching")

    # Live play-by-play
    plays = []
    for play in feed["playByPlay"]["allPlays"]:
        a    = play["about"]
        desc = play["result"].get("description","")
        t    = parser.isoparse(a["startTime"]).astimezone(tz.tzlocal()).strftime("%I:%M %p")
        plays.append({
            "Inning": a["inning"],
            "Half":   a["halfInning"],
            "Time":   t,
            "Desc":   desc
        })
    show_stats(plays, f"Live Play-by-Play ({len(plays)} events)")

else:
    # FALLBACK: linescore & boxscore endpoints
    print("Live data unavailable—using boxscore & linescore endpoints.\n")
    
    # Linescore fallback
    ls_fb = requests.get(f"https://statsapi.mlb.com/api/v1/game/{game_pk}/linescore").json()
    df_fb = (
        pd.DataFrame(ls_fb["innings"])[["num","away","home"]]
          .rename(columns={"num":"Inning","away":"Away","home":"Home"})
    )
    df_fb["Away Total"] = ls_fb["teams"]["away"]["runs"]
    df_fb["Home Total"] = ls_fb["teams"]["home"]["runs"]
    show_linescore(df_fb, "Fallback")

    # Boxscore fallback
    box_fb = requests.get(f"https://statsapi.mlb.com/api/v1/game/{game_pk}/boxscore").json()

    def lookup_player(pid, side):
        # try top‐level players, then per‐team players
        top = box_fb.get("players", {})
        if f"ID{pid}" in top:
            return top[f"ID{pid}"]
        return box_fb["teams"][side].get("players", {}).get(f"ID{pid}", {})

    # Batting fallback
    bat_fb = []
    for side in ("home","away"):
        team = box_fb["teams"][side]
        for pid in team.get("batters",[]):
            p = lookup_player(pid, side)
            stats = p.get("stats",{}).get("batting",{})
            if stats:
                bat_fb.append({
                    "Team": team["team"]["name"],
                    "Player": p.get("person",{}).get("fullName",""),
                    **stats
                })
    show_stats(bat_fb, "Fallback Batting")

    # Pitching fallback
    pit_fb = []
    for side in ("home","away"):
        team = box_fb["teams"][side]
        for pid in team.get("pitchers",[]):
            p = lookup_player(pid, side)
            stats = p.get("stats",{}).get("pitching",{})
            if stats:
                pit_fb.append({
                    "Team": team["team"]["name"],
                    "Player": p.get("person",{}).get("fullName",""),
                    **stats
                })
    show_stats(pit_fb, "Fallback Pitching")


Live data unavailable—using boxscore & linescore endpoints.

Fallback Linescore


,Inning,Away,Home,Away Total,Home Total
0,1,"{'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBas...","{'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBas...",0,0


Fallback Batting


,Team,Player,summary,gamesPlayed,flyOuts,groundOuts,airOuts,runs,doubles,triples,...,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,atBatsPerHomeRun,popOuts,lineOuts
0,Arizona Diamondbacks,Corbin Carroll,0-1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,-.--,1,0
1,Arizona Diamondbacks,Geraldo Perdomo,0-1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
2,Arizona Diamondbacks,Pavin Smith,0-1 | K,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
3,Arizona Diamondbacks,Josh Naylor,0-0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
4,Arizona Diamondbacks,Eugenio Suárez,0-0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
5,Arizona Diamondbacks,Lourdes Gurriel Jr.,0-0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
6,Arizona Diamondbacks,James McCann,0-0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
7,Arizona Diamondbacks,Alek Thomas,0-0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
8,Arizona Diamondbacks,Blaze Alexander,0-0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
9,Kansas City Royals,Jonathan India,0-1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0


Fallback Pitching


,Team,Player,summary,gamesPlayed,gamesStarted,flyOuts,groundOuts,airOuts,runs,doubles,...,runsScoredPer9,homeRunsPer9,inheritedRunners,inheritedRunnersScored,catchersInterference,sacBunts,sacFlies,passedBall,popOuts,lineOuts
0,Arizona Diamondbacks,Ryne Nelson,"1.0 IP, 0 ER, K, 0 BB",1,1,1,1,1,0,0,...,0.00,0.00,0,0,0,0,0,0,0,0
1,Kansas City Royals,Michael Wacha,"1.0 IP, 0 ER, K, 0 BB",1,1,0,1,1,0,0,...,0.00,0.00,0,0,0,0,0,0,1,0
